In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime, timedelta
import re
import time

from scipy.stats import skew
from scipy import stats
from statsmodels.formula.api import ols
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("../input"))


In [ ]:
train = pd.read_excel("../input/Train.xlsx")
test = pd.read_excel("../input/Test.xlsx")

## Quick Overview of Data

In [ ]:
train.head(2)

In [ ]:
test.head(5)

In [ ]:
# check that all the data was properly imported
print('shape:', train.shape, '\n')
print('dtpyes: \n' + str(train.dtypes))

In [ ]:
# Get counts of categorical features
print('number of airlines:', len(train['Airline'].unique()))
print('number of sources:', len(train['Source'].unique()))
print('number of destinations:', len(train['Destination'].unique()))
print('number of stops:', len(train['Total_Stops'].unique()))
print('number of Additional Info:', len(train['Additional_Info'].unique()))

In [ ]:
train['Date_of_Journey'] = pd.to_datetime(train['Date_of_Journey'],format="%d/%m/%Y")
test['Date_of_Journey'] = pd.to_datetime(test['Date_of_Journey'],format="%d/%m/%Y")

In [ ]:
train['Dep_Time'] = pd.to_datetime(train['Dep_Time'],format="%H:%M")
test['Dep_Time'] = pd.to_datetime(test['Dep_Time'],format="%H:%M")

In [ ]:
# Check range of dates in df
print('Date range:', train['Date_of_Journey'].max() - train['Date_of_Journey'].min())
print('Latest Date:', train['Date_of_Journey'].max())
print('Earliest Date:', train['Date_of_Journey'].min())
print(train['Date_of_Journey'].describe())

In [ ]:
# Price analysis
print(train['Price'].describe())

In [ ]:
train.isnull().sum()

In [ ]:
#There is one record in train data which have missing value in route an stops. we will delete this
train.dropna(axis=0,inplace= True)
train.reset_index(inplace= True)
train.drop('index', axis=1, inplace= True)

In [ ]:
train.head(2)

In [ ]:
train.shape, test.shape

## Preprocessing

-  Comvert the duration column to hours or minutes
- Delete arrival time, Few recors looks off  as it is not in sync with date of journey and departure time. We will calculate the arrival time based on the departure and duration
- Explore the various categorical feature 
- Add some time and duration based feature

In [ ]:
# We already have duration variable . lets delete Arrival time
train.drop('Arrival_Time', axis=1, inplace= True)
test.drop('Arrival_Time', axis=1, inplace= True)

### Feature Engineering

In [ ]:
train['day_d'] = train.Date_of_Journey.dt.day
train['month_d'] = train.Date_of_Journey.dt.month
train['weekday_d'] = train.Date_of_Journey.dt.weekday
train['month_start'] = train.Date_of_Journey.dt.is_month_start
train['month_end'] = train.Date_of_Journey.dt.is_month_end


test['day_d'] = test.Date_of_Journey.dt.day
test['month_d'] = test.Date_of_Journey.dt.month
test['weekday_d'] = test.Date_of_Journey.dt.weekday
test['month_start'] = test.Date_of_Journey.dt.is_month_start
test['month_end'] = test.Date_of_Journey.dt.is_month_end

In [ ]:
train['day_offset'] = (train['Date_of_Journey'] - datetime(2019, 3, 1))
test['day_offset'] = (test['Date_of_Journey'] - datetime(2019, 3, 1))

train['day_offset'] = train['day_offset'].dt.days
test['day_offset'] = test['day_offset'].dt.days

train['day_offset'] = train['day_offset']+1
test['day_offset'] = test['day_offset']+1

In [ ]:
for i in range(train.shape[0]):
    if train.ix[i,"Duration"].find('m') < 0:
        train.ix[i,"Duration_final"] = int(re.findall('\dh|\d\dh',train.ix[i,"Duration"])[0].strip('h'))*60
    else:
        train.ix[i,"Duration_final"] = int(re.findall('\dh|\d\dh',train.ix[i,"Duration"])[0].strip('h'))*60 + int(re.findall('\dm|\d\dm',train.ix[i,"Duration"])[0].strip('m')) 

In [ ]:
for i in range(test.shape[0]):
    if test.ix[i,"Duration"].find('m') < 0:
        test.ix[i,"Duration_final"] = int(re.findall('\dh|\d\dh',test.ix[i,"Duration"])[0].strip('h'))*60
    else:
        test.ix[i,"Duration_final"] = int(re.findall('\dh|\d\dh',test.ix[i,"Duration"])[0].strip('h'))*60 + int(re.findall('\dm|\d\dm',test.ix[i,"Duration"])[0].strip('m')) 

In [ ]:
#we already have duration in minute so we will delete this text column
train.drop(['Duration'], axis=1, inplace= True)
test.drop(['Duration'], axis=1, inplace= True)

In [ ]:
for i in range(train.shape[0]):
    train.ix[i,"Date_of_departure_ts"] = pd.datetime(train.Date_of_Journey.dt.year[i],
                                                     train.Date_of_Journey.dt.month[i],
                                                     train.Date_of_Journey.dt.day[i],
                                                     train.Dep_Time.dt.hour[i],
                                                     train.Dep_Time.dt.minute[i])

for i in range(test.shape[0]):
    test.ix[i,"Date_of_departure_ts"] = pd.datetime(test.Date_of_Journey.dt.year[i],
                                                     test.Date_of_Journey.dt.month[i],
                                                     test.Date_of_Journey.dt.day[i],
                                                     test.Dep_Time.dt.hour[i],
                                                     test.Dep_Time.dt.minute[i])

In [ ]:
train.drop(['Dep_Time'], axis=1, inplace= True)
test.drop(['Dep_Time'], axis=1, inplace= True)

train.drop(['Date_of_Journey'], axis=1, inplace= True)
test.drop(['Date_of_Journey'], axis=1, inplace= True)

In [ ]:
# Create arrival time stamp based on departure time and duration final

for i in range(train.shape[0]):
    train.ix[i,"Arrival_time_ts"] = train.Date_of_departure_ts[i] + timedelta(minutes = train.Duration_final[i])

for i in range(test.shape[0]):
    test.ix[i,"Arrival_time_ts"] = test.Date_of_departure_ts[i] + timedelta(minutes = test.Duration_final[i])


In [ ]:
train.head(1)

In [ ]:
## creating features based on arrival time
train['day_a'] = train.Arrival_time_ts.dt.day
test['day_a'] = test.Arrival_time_ts.dt.day

train['hour_d'] = train.Date_of_departure_ts.dt.hour
test['hour_d'] = test.Date_of_departure_ts.dt.hour

train['hour_a'] = train.Arrival_time_ts.dt.hour
test['hour_a'] = test.Arrival_time_ts.dt.hour

In [ ]:
train['is_arrival_same_day'] = train['day_d'] == train['day_a']
test['is_arrival_same_day'] = test['day_d'] == test['day_a']

In [ ]:
train.drop(['Date_of_departure_ts','Arrival_time_ts'], axis=1, inplace= True)
test.drop(['Date_of_departure_ts','Arrival_time_ts'], axis=1, inplace= True)

In [ ]:
train.head(1)

### Data Cleaning

In [ ]:
train['Total_Stops'] = train['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4})
test['Total_Stops'] = test['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4})

In [ ]:
train.loc[train.Airline =='Jet Airways Business', 'Additional_Info'] = 'Business class'
test.loc[test.Airline =='Jet Airways Business', 'Additional_Info'] = 'Business class'

train.loc[train.Airline =='Jet Airways Business', 'Airline'] = 'Jet Airways'
test.loc[test.Airline =='Jet Airways Business', 'Airline'] = 'Jet Airways'

train.loc[train.Airline =='Multiple carriers Premium economy', 'Additional_Info'] = 'Premium economy'
test.loc[test.Airline =='Multiple carriers Premium economy', 'Additional_Info'] = 'Premium economy'

train.loc[train.Airline =='Vistara Premium economy', 'Additional_Info'] = 'Premium economy'
test.loc[test.Airline =='Vistara Premium economy', 'Additional_Info'] = 'Premium economy'

train.loc[train.Airline =='Multiple carriers Premium economy', 'Airline'] = 'Multiple carriers'
test.loc[test.Airline =='Multiple carriers Premium economy', 'Airline'] = 'Multiple carriers'

train.loc[train.Airline =='Vistara Premium economy', 'Airline'] = 'Vistara'
test.loc[test.Airline =='Vistara Premium economy', 'Airline'] = 'Vistara'

train.loc[train.Destination =='New Delhi', 'Destination'] = 'Delhi'
test.loc[test.Destination =='New Delhi', 'Destination'] = 'Delhi'

In [ ]:
train['month_start'] = train['month_start'].map({False:0,True:1})                                                           
test['month_start'] = test['month_start'].map({False:0,True:1})

train['month_end'] = train['month_end'].map({False:0,True:1})                                                              
test['month_end'] = test['month_end'].map({False:0,True:1})

train['is_arrival_same_day'] = train['is_arrival_same_day'].map({False:0,True:1})                                                              
test['is_arrival_same_day'] = test['is_arrival_same_day'].map({False:0,True:1})

## More Exploration

In [ ]:
# Plot Histogram
sns.distplot(train['Price'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['Price'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')

fig = plt.figure()
res = stats.probplot(train['Price'], plot=plt)
plt.show()

print("Skewness: %f" % train['Price'].skew())
print("Kurtosis: %f" % train['Price'].kurt())

In [ ]:
# Correlation Matrix Heatmap
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=1, square=True, annot= True);

In [ ]:
# Overall Quality vs Sale Price
var = 'Airline'
data = pd.concat([train['Price'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 8))
fig = sns.boxplot(x=var, y="Price", data=data)
fig.axis(ymin=0, ymax=90000);

In [ ]:
# Living Area vs Sale Price
sns.jointplot(x=train['Duration_final'], y=train['Price'], kind='reg')

In [ ]:
# Living Area vs Sale Price
sns.jointplot(x=train['day_offset'], y=train['Price'], kind='reg')

In [ ]:
train['Route_list'] = train.Route.str.split(" → ")
test['Route_list'] = test.Route.str.split(" → ")

# adding a feature to check if flight have the stops in metro (BOM,DEL,BLR,MAA,CCU)

In [ ]:
for i in range(train.shape[0]):
    if train.Total_Stops[i] == 0:
        train.ix[i,"is_stop_BOM"] = 'False'
    else:
        train.ix[i,"is_stop_BOM"] = str('BOM' in train.Route_list[i][1:(train.Total_Stops[i]+1)])
        
for i in range(test.shape[0]):
    if test.Total_Stops[i] == 0:
        test.ix[i,"is_stop_BOM"] = 'False'
    else:
        test.ix[i,"is_stop_BOM"] = str('BOM' in test.Route_list[i][1:(test.Total_Stops[i]+1)])

In [ ]:
for i in range(train.shape[0]):
    if train.Total_Stops[i] == 0:
        train.ix[i,"is_stop_DEL"] = 'False'
    else:
        train.ix[i,"is_stop_DEL"] = str('DEL' in train.Route_list[i][1:(train.Total_Stops[i]+1)])
        
for i in range(test.shape[0]):
    if test.Total_Stops[i] == 0:
        test.ix[i,"is_stop_DEL"] = 'False'
    else:
        test.ix[i,"is_stop_DEL"] = str('DEL' in test.Route_list[i][1:(test.Total_Stops[i]+1)])

In [ ]:
for i in range(train.shape[0]):
    if train.Total_Stops[i] == 0:
        train.ix[i,"is_stop_BLR"] = 'False'
    else:
        train.ix[i,"is_stop_BLR"] = str('BLR' in train.Route_list[i][1:(train.Total_Stops[i]+1)])
        
for i in range(test.shape[0]):
    if test.Total_Stops[i] == 0:
        test.ix[i,"is_stop_BLR"] = 'False'
    else:
        test.ix[i,"is_stop_BLR"] = str('BLR' in test.Route_list[i][1:(test.Total_Stops[i]+1)])

In [ ]:
for i in range(train.shape[0]):
    if train.Total_Stops[i] == 0:
        train.ix[i,"is_stop_MAA"] = 'False'
    else:
        train.ix[i,"is_stop_MAA"] = str('MAA' in train.Route_list[i][1:(train.Total_Stops[i]+1)])
        
for i in range(test.shape[0]):
    if test.Total_Stops[i] == 0:
        test.ix[i,"is_stop_MAA"] = 'False'
    else:
        test.ix[i,"is_stop_MAA"] = str('MAA' in test.Route_list[i][1:(test.Total_Stops[i]+1)])

In [ ]:
for i in range(train.shape[0]):
    if train.Total_Stops[i] == 0:
        train.ix[i,"is_stop_CCU"] = 'False'
    else:
        train.ix[i,"is_stop_CCU"] = str('CCU' in train.Route_list[i][1:(train.Total_Stops[i]+1)])
        
for i in range(test.shape[0]):
    if test.Total_Stops[i] == 0:
        test.ix[i,"is_stop_CCU"] = 'False'
    else:
        test.ix[i,"is_stop_CCU"] = str('CCU' in test.Route_list[i][1:(test.Total_Stops[i]+1)])

In [ ]:
train.drop(['Route','Route_list'], axis=1, inplace= True)
test.drop(['Route','Route_list'], axis=1, inplace= True)

In [ ]:
train['is_stop_CCU'] = train['is_stop_CCU'].map({'False':0,'True':1})                                                           
test['is_stop_CCU'] = test['is_stop_CCU'].map({'False':0,'True':1})

train['is_stop_BOM'] = train['is_stop_BOM'].map({'False':0,'True':1})                                                           
test['is_stop_BOM'] = test['is_stop_BOM'].map({'False':0,'True':1})

train['is_stop_MAA'] = train['is_stop_MAA'].map({'False':0,'True':1})                                                           
test['is_stop_MAA'] = test['is_stop_MAA'].map({'False':0,'True':1})

train['is_stop_DEL'] = train['is_stop_DEL'].map({'False':0,'True':1})                                                           
test['is_stop_DEL'] = test['is_stop_DEL'].map({'False':0,'True':1})

train['is_stop_BLR'] = train['is_stop_BLR'].map({'False':0,'True':1})                                                           
test['is_stop_BLR'] = test['is_stop_BLR'].map({'False':0,'True':1})

## Statistical test to check the significance of additional info

In [ ]:
train['Additional_Info'] = train['Additional_Info'].map({'No info':'No info', 
                                                               "In-flight meal not included":"In-flight meal not included",
                                                               'No Info':'No info',
                                                               '1 Short layover':'Layover',
                                                               '1 Long layover':'Layover',
                                                               '2 Long layover':'Layover',
                                                               'Business class':'Business class',
                                                               'No check-in baggage included':'No check-in baggage included',
                                                               'Change airports':'Change airports',
                                                               'Red-eye flight':'No info'})
                                                               
test['Additional_Info'] = test['Additional_Info'].map({'No info':'No info', 
                                                               "In-flight meal not included":"In-flight meal not included",
                                                               '1 Long layover':'Layover',
                                                               'Business class':'Business class',
                                                               'No check-in baggage included':'No check-in baggage included',
                                                               'Change airports':'Change airports'})
                                                               



In [ ]:
# Additional_Info
anova = ols('Price ~ C(Additional_Info)', data=train).fit()
anova.summary()

In [ ]:
train.loc[train.Airline == 'Trujet', 'Airline'] = 'IndiGo'

In [ ]:
# train.loc[train.Price > 35000, 'Additional_Info'] = 'Business class'

In [ ]:
train.head(2)

In [ ]:
test.head(5)

### Adding a feature which shows how many minutes extra a flight takes than the Usual non stop flight

In [ ]:
max_dur = train[train.Total_Stops == 0][['Source','Destination','Duration_final']].groupby(['Source','Destination']).max().reset_index()
max_dur

In [ ]:
index_train = np.arange(1,train.shape[0]+1,1)
train['index'] = index_train
train.head(5)

In [ ]:
index_test = np.arange(1,test.shape[0]+1,1)
test['index'] = index_test
test.head(5)

In [ ]:
train = train.merge(max_dur, on = ['Source','Destination'])
test = test.merge(max_dur, on = ['Source','Destination'])

In [ ]:
train = train.sort_values(by = 'index')
train.drop('index', axis=1, inplace= True)
train.head(5)

In [ ]:
test = test.sort_values(by = 'index')
test.drop('index', axis=1, inplace= True)
test.head(5)

In [ ]:
train['duration_diff'] = train['Duration_final_x'] - train['Duration_final_y']
test['duration_diff'] = test['Duration_final_x'] - test['Duration_final_y']

In [ ]:
# train.loc[train.duration_diff <0,'duration_diff'] = 0
# test.loc[train.duration_diff <0,'duration_diff'] = 0

In [ ]:
train.drop('Duration_final_y', axis=1, inplace= True)
test.drop('Duration_final_y', axis=1, inplace= True)

In [ ]:
# train.drop('Additional_Info', axis=1, inplace= True)
# test.drop('Additional_Info', axis=1, inplace= True)

In [ ]:
test.head(5)

In [ ]:
train.to_csv('train_processed.csv', index = False)
test.to_csv('test_processed.csv', index = False)

In [ ]:
train_df = pd.get_dummies(train)
test_df = pd.get_dummies(test)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head(3)

## Modelling

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [ ]:
y_train = np.log(train_df.Price.values)
# y_train = train_df.Price.values
X_train = train_df.drop('Price', axis=1)

### LightGBM 

In [ ]:
# LightGBM
folds = KFold(n_splits=10, shuffle=False, random_state=2139)

oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(test_df.shape[0])


valid_score = 0


for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("Fold idx:{}".format(n_fold + 1))
    trn_x, trn_y = X_train.iloc[trn_idx], y_train[trn_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train[val_idx]    
    
    train_data = lgb.Dataset(data=trn_x, label=trn_y)
    valid_data = lgb.Dataset(data=val_x, label=val_y)   
    
    params = {
        'objective': 'regression',
        "metric" : "rmse",
        'nthread': 4,
        'num_leaves': 13,
        'learning_rate': 0.01,
        'max_depth': 7,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'scale_pos_weight':1,
        'num_rounds':50000,
        'early_stopping_rounds':1000,
    }
    
#     params = {"objective" : "regression",
#               "metric" : "rmse",
#               'n_estimators':30000,
#               'max_depth':7, 
#               'early_stopping_rounds':500,
#               "num_leaves" : 13,
#               "learning_rate" : 0.01,
#               "bagging_fraction" : 0.9,
#               "bagging_seed" : 0,
#               "num_threads" : 4,
#               "colsample_bytree" : 0.7
#              }
    
    lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=1000) 
    oof_preds[val_idx] = lgb_model.predict(val_x, num_iteration=lgb_model.best_iteration)
    sub_pred = lgb_model.predict(test_df, num_iteration=lgb_model.best_iteration) 
    sub_preds += sub_pred/ folds.n_splits


In [ ]:
r2_score(y_train,lgb_model.predict(X_train))

In [ ]:
final_sub = np.exp(sub_preds)

In [ ]:
#Predict from test set
# prediction = model.predict(test, num_iteration = model.best_iteration)      
submission1 = pd.DataFrame({
        "Price": final_sub,
})

submission1.to_excel('submission_lgb.xlsx',index=False)
submission1.head()

In [ ]:
feature_importance = pd.DataFrame({"columns":X_train.columns, "Value":lgb_model.feature_importance()}).sort_values(by = 'Value', ascending = False)
feature_importance.head(15)

In [ ]:
submission1.describe()

In [ ]:
submission1.head()

### Catboost

In [ ]:
# Training and Validation Set
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=23)

In [ ]:
# Train Model
print("Train CatBoost Decision Tree")
modelstart= time.time()
cb_model = CatBoostRegressor(iterations=100000,
                             learning_rate=0.01,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 23,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)


In [ ]:
X_train.columns

In [ ]:
cat_features = []

In [ ]:
cb_model.fit(X_tr, y_tr,eval_set=(X_val,y_val),use_best_model=True,verbose=None, verbose_eval=75)

In [ ]:
print("Model Evaluation Stage")
print(cb_model.get_params())
print('RMSE:', np.sqrt(mean_squared_error(y_val, cb_model.predict(X_val))))

In [ ]:
cb_sub = np.exp(cb_model.predict(test_df))

#Predict from test set
# prediction = model.predict(test, num_iteration = model.best_iteration)      
submission2 = pd.DataFrame({
        "Price": cb_sub,
})

submission2.to_excel('submission_cb.xlsx',index=False)
submission2.head()

In [ ]:
submission2.describe()